In [128]:
#this code downloads ESAs from section 52 applications
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import wget
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [129]:
#folder with list of missing ESA folder
#df of missing projects
missing = pd.read_csv('F:/Environmental Baseline Data/Version 2/Support files/list_of_Section_52_projects.csv', 
                      encoding='ISO-8859-1')
missing.head()
missing.shape
missing['Application title'].nunique()

,Section,Application title,link to folder with ESA,Folder link,Company Name
0,52,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,268693,EnCana Ekwan Pipeline Inc.
1,52,2006-05-23 - Application for the Brunswick Pip...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,408713,Emera Brunswick Pipeline Company Ltd.
2,52,"2007-03-09 - Application for Line 13 Transfer,...",https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,456315,Enbridge Southern Lights GP Inc.
3,52,2007-06-28 - Application for the Line 4 Extens...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,467952,Enbridge Pipelines Inc.
4,52,2008-02-27 - Application to construct and oper...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/5...,500494,Westcoast Energy Inc.


(67, 5)

36

In [130]:
#list of links 
urls_list = missing['link to folder with ESA'].tolist()
urls_list_load = [str(x).replace('https://apps.cer-rec.gc.ca/REGDOCS/Item/View/', 
                                 'https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/') for x in urls_list]

In [131]:
all_esa = []

for x in urls_list_load:
    cookies = {'RDI-NumberOfRecords': '200'}
    r = requests.get(x, cookies = cookies)
    soup = bs(r.text, 'lxml')
    tds = soup.find_all('td', {'class' : 'nopadding break-anywhere'})
    file_name = [x.text for x in soup.find_all('a', {'class' : 'row-icon'})]
    hrefs = [a['href'] for a in soup.find_all('a', {'class' : 'row-icon'})]
    df = pd.DataFrame({'file_name' : file_name, 'hrefs' : hrefs, 'folder' : str(x)})
    all_esa.append(df)

df = pd.concat(all_esa, axis = 0)
df = df[df['file_name'].str.contains('ESA|nvironment|Horn|ocio|EIA|V14|V15|V16|V17|V18|V19|V20|V21|V22|V23|V24|V25|A1X1|A1C3|A3S1|A3S2|Goldboro')]
df['Folder link'] = df['folder'].str.replace('https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/', '').astype(int)
df = df.merge(missing, on = 'Folder link')
postal_series = df['file_name'].str.extract('(A\d\D\d\D\d)')
df.insert(1, 'postal', postal_series)
# df['Folder link'] = df['Folder link']
df.shape

(1331, 9)

In [132]:
#check to ensure all ESPs are captured; should return an empty; if project name, ESAs not captured
df_list = df['Application title'].unique()
missing_list = missing['Application title'].unique()

print(list(set(missing_list) - set(df_list)))


[]


In [133]:
# leave for future in case need to take Projects out of corpus
# remove three projects b/c they already exist in original corpus (ie. ESA data is already in corpus)
# searchfor = ['Groundbirch', 'Towerbirch', 'GH-001-2012']
# df = df[~df['Application title'].str.contains('|'.join(searchfor))]
# df.shape

In [134]:
#clean up href link
df['hrefs'] = df['hrefs'].str.replace('/REGDOCS', 'https://apps.cer-rec.gc.ca/REGDOCS')
download_ESA_links = df['hrefs']
len(download_ESA_links)
df.columns

1331

Index(['file_name', 'postal', 'hrefs', 'folder', 'Folder link', 'Section',
       'Application title', 'link to folder with ESA', 'Company Name'],
      dtype='object')

In [135]:
#downloads all ESA PDFs from list 
# for x in download_links:
#     wget.download(x, 'F:\Environmental Baseline Data\Version 2\Data\PDFs')

In [136]:
df.head(2)

,file_name,postal,hrefs,folder,Folder link,Section,Application title,link to folder with ESA,Company Name
0,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,A0H8C0,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,268693,52,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,EnCana Ekwan Pipeline Inc.
1,A0H8C1 - 13.0 EIA - Section 13.7 Wildlife Part 1,A0H8C1,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,268693,52,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,EnCana Ekwan Pipeline Inc.


In [137]:
#regular expressions to parse out application date and hearing order 
application_date = df['Application title'].str.extract('(\d\d\d\d-\d\d-\d\d)')
df.insert(7, 'application_date', application_date)

hearing_order = df['Application title'].str.extract(r'(\D\D-\d(.+?)\d\d\d\d)')
hearing_order = hearing_order.drop([1], axis=1)
df.insert(8, 'hearing_order', hearing_order)

In [141]:
#check to ensure regex worked
hearing_order.isnull().values.any()
application_date.isnull().values.any()

False

False

In [139]:
df.shape
df.columns
df['Application title'].nunique()

(1331, 11)

Index(['file_name', 'postal', 'hrefs', 'folder', 'Folder link', 'Section',
       'Application title', 'application_date', 'hearing_order',
       'link to folder with ESA', 'Company Name'],
      dtype='object')

36

In [140]:
df.to_csv('ESA_Index1.csv')